In [ ]:
import numpy as np
import scipy.linalg as la
import math
import matplotlib.pyplot as plt
import time
import random

from utils import *

from numba import jit
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.datasets import load_svmlight_file

dataset = "mushrooms.txt"
data = load_svmlight_file(dataset)
A, b = data[0].toarray(), data[1]

b = 2 * b - 3
n, d = A.shape

In [ ]:
from sklearn.model_selection import train_test_split
A_train, A_test, b_train, b_test = train_test_split(A, b, train_size=0.8001, shuffle=True)
n_train = A_train.shape[0]

In [ ]:
# parameters
beta_x = 0.1
beta_y = 0.1
C = 0.1

L_P = 5.25
L_Q = 2 * min(beta_x, beta_y)
mu_Q = L_Q

In [ ]:
from numba import jit
import warnings
warnings.filterwarnings("ignore")


@jit
def Pi(z, i):
    x, y = split(z)
    px = (-b_train[i] * (A_train[i].reshape((d, 1)) + y)) / (1 + np.exp(b_train[i] * float(x.T @ (A_train[i].reshape((d, 1)) + y))))
    py = (b_train[i] * x) / (1 + np.exp(b_train[i] * float(x.T @ (A_train[i].reshape((d, 1)) + y))))
    return merge(px, py).reshape((2*d,1))

@jit
def P(z):
    p = np.zeros((2*d, 1))
    ind = [i for i in range(0, 6500, 1)]
    for i in ind:
        p += Pi(z, i)
    return p/n_train

@jit
def Q(z):
    x, y = split(z)
    return merge(beta_x * x, beta_y * y).reshape((2*d, 1))

def R(z):
    return P(z) + Q(z)

def err_norm(z):
    return np.linalg.norm(R(z))**2

In [ ]:
def proj(z):
    x, y = split(z)
    y = y * C / np.linalg.norm(y, ord=2)
    return merge(x, y).reshape((2*d, 1))

def extragradient(z0, gamma, error=err_norm, iters=3000):

    z = z0
    times = [0.]
    errs = [error(z)]
    start = time.time()

    for i in range(0, iters-1):
        
        w = proj(z - gamma * R(z))
        z = proj(z - gamma * R(w))

        times.append(time.time() - start)
        errs.append(error(z))

    return z, times, errs

In [ ]:
L_B = 2 * L_P + L_Q
iters_eagv = math.ceil(9 * L_B / L_P) - 1
iters_eagv = iters_eagv // 5
print(iters_eagv)

In [ ]:
def create_B(z_k, theta):
    def B(z):
        return P(z_k) + Q(z) + (z - z_k)/theta
    return B


def EAG_V(z0, B, iters=iters_eagv):

    z = z0
    gamma = 0.618/L_B

    for i in range(0, iters-1):
        
        w = z + (z0 - z)/(i+2) - gamma * B(z)
        z = z + (z0 - z)/(i+2)- gamma * B(w)
        gamma = gamma * (1 - (gamma * L_B)**2 / ((i+1)*(i+3)*(1 - (gamma * L_B)**2)) )

    return z


def extragrad_sliding(z0, gamma, theta, error=err_norm, iters_inner=iters_eagv, iters=3000):

    z = z0
    z_vals = [z]
    times = [0.]
    errs = [error(z)]
    start = time.time()

    for i in range(0, iters-1):
        
        B_tmp = create_B(z, theta)
        u = EAG_V(z, B_tmp, iters_inner)
        z = proj(z - gamma * R(u))

        z_vals.append(z)
        times.append(time.time() - start)
        errs.append(error(z))

    return z_vals, times, errs

In [ ]:
# Starting point
x0, y0 = np.random.uniform(low=-L_P/d, high=L_P/d, size=(d,1)), np.random.uniform(low=-1, high=1, size=(d,1))
z0 = proj(merge(x0, y0).reshape((2*d, 1)))

In [ ]:
# Optimal params
gamma_opt = 6.5/L_P
theta_opt = 0.6/L_P
iters_eagv = 3
iters = 30

_, times1, errs1 = extragradient(z0, gamma=1/L_P, error=err_norm, iters=iters)
_, times2, errs2 = extragrad_sliding(z0, gamma=gamma_opt, theta=theta_opt, iters_inner=iters_eagv, 
                                     error=err_norm, iters=iters)

In [ ]:
# Criterion from iteration

plt.figure(figsize = (4.5,3.5))
plt.semilogy(range(len(errs1)), errs1, color='g', label='EG')
plt.semilogy(range(len(errs2)), errs2, color='r', label='EG Sliding')

plt.title('Criterion from iteration')
plt.xlabel("Num iteration")
plt.ylabel(r"Log $\|R(z)\|_2^2$")

plt.grid(True, linestyle='--', linewidth=0.4)

plt.legend(fontsize=8)
plt.tight_layout()
plt.savefig(f'Criterion_from_iteration_logloss.pdf')
plt.show()

In [ ]:
# Criterion from time 

plt.figure(figsize = (4.5,3.5))
plt.semilogy(times1, errs1, color='g', label='EG')
plt.semilogy(times2[:15], errs2[:15], color='r', label='EG Sliding')

plt.title('Criterion from time')
plt.xlabel("Computation time, s")
plt.ylabel(r"Log $\|R(z)\|_2^2$")

plt.grid(True, linestyle='--', linewidth=0.4)

plt.legend(fontsize=8)
plt.tight_layout()
plt.savefig(f'Criterion_from_time_logloss.pdf')
plt.show()